## Notebook to combine a1355e and 3ada8b RT files into one spreadsheet for Kurt 

### David.J.Caldwell 7.14.2018



In [1]:

import numpy as np
import scipy
from scipy.io import loadmat
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns; sns.set()

sns.set_context("poster") # this makes things large 

%matplotlib inline



In [3]:
### decide whether or not to save figures and data for MATLAB

saveFig = True
saveMat = True

In [5]:
### import the .mat data files
mat1 = loadmat('a1355e_compareResponse_block_1_changePts_noDelay',squeeze_me=1)
mat2 = loadmat('a1355e_compareResponse_block_3_changePts_noDelay',squeeze_me=1)
mat3 = loadmat('a1355e_compareResponse_block_4_changePts_noDelay',squeeze_me=1)
mat4 = loadmat('3ada8b_compareResponse_block_1_changePts_noDelay',squeeze_me=1)
mat5 = loadmat('3ada8b_compareResponse_block_2_changePts_noDelay',squeeze_me=1)

In [6]:
### extract variables of interest
### respLo = the minimum response time
### respHi = the maximum response time

respLo = 150 # minimum response time - 150 ms
respHi = 1000 # maximum response time - 1000 ms

button1 = mat1['buttonLocs']
button2 = mat2['buttonLocs']
button3 = mat3['buttonLocs']
button4 = mat4['buttonLocs']
button5 = mat5['buttonLocs']


In [18]:
### prepare for pandas data frame

a = np.arange(0,button1.shape[0])
ndata1 = {n: 1e3*button1[n] for n in a}
a = np.arange(0,button2.shape[0])
ndata2 = {n: 1e3*button2[n] for n in a}
a = np.arange(0,button3.shape[0])
ndata3 = {n: 1e3*button3[n] for n in a}
a = np.arange(0,button4.shape[0])
ndata4 = {n: 1e3*button4[n] for n in a}
a = np.arange(1,button5.shape[0])
ndata5 = {n: 1e3*button5[n] for n in a} # ignore null from this data file

In [19]:
ndata5

{1: array([435.03616, 285.77792, 243.42528, 260.38272, 219.09504, 279.7568 ,
        411.68896, 260.5056 , 371.3024 , 221.10208, 230.31808, 247.56224,
        286.06464, 236.01152, 308.79744, 255.54944, 279.552  ]),
 2: array([522.89536, 430.85824, 395.59168, 422.95296, 483.57376, 376.34048,
        471.16288, 425.5744 , 404.39808, 479.10912, 410.0096 , 403.41504,
        441.9584 , 404.02944, 388.3008 , 255.54944, 279.552  ])}

In [20]:
### make pandas dataframes

dataFdict1 = pd.DataFrame.from_dict(ndata1,orient='columns')
dataFdict2 = pd.DataFrame.from_dict(ndata2,orient='columns')
dataFdict3 = pd.DataFrame.from_dict(ndata3,orient='columns')
dataFdict4 = pd.DataFrame.from_dict(ndata4,orient='columns')
dataFdict5 = pd.DataFrame.from_dict(ndata5,orient='columns')


In [28]:
dataFdict1

,0,1,2,3,4,5,6
0,372.16256,NaN,NaN,420.37248,NaN,508.06784,682.84416
1,340.29568,NaN,NaN,542.80192,843.93984,393.91232,629.43232
2,265.91232,NaN,NaN,463.05280,NaN,719.13472,744.40704
3,208.11776,NaN,NaN,439.17312,NaN,429.58848,NaN
4,171.29472,NaN,NaN,315.43296,NaN,381.25568,NaN
5,308.14208,NaN,NaN,280.00256,NaN,423.48544,NaN
6,346.39872,NaN,NaN,326.53312,NaN,429.17888,483.86048
7,245.51424,NaN,NaN,425.45152,NaN,320.63488,NaN
8,271.89248,NaN,NaN,418.20160,NaN,470.22080,980.09088
9,540.91776,NaN,NaN,292.82304,NaN,432.08704,NaN


In [29]:
### assign columns for dataframes

### a1355e

dataFdict1.columns = ['haptic','null','off-target','2000 uA 100 ms','1000 uA 200 ms','1500 uA 200 ms','1000 uA 300 ms']
dataFdict2.columns = ['constant train','high initial pulses']
dataFdict3.columns = ['constant train','high initial pulses']

dataMelt1 = pd.melt(dataFdict1)
dataMelt1['Subject'] = '1'
dataMelt1['block'] = '1'

dataMelt2 = pd.melt(dataFdict2)
#dataMelt1['Subject'] = 'a1355e'
dataMelt2['Subject'] = '1'
dataMelt2['block'] = '2'
               
dataMelt3 = pd.melt(dataFdict3)
#dataMelt2['Subject'] = 'a1355e'
dataMelt3['Subject'] = '1'
dataMelt3['block'] = '3'

### 3ada8b
### assign columns for dataframes

dataFdict4.columns = ['haptic','null','off-target','1250 uA','800 uA','3000 uA','2 @ 3000 uA, 38 @ 1250 uA','2 @ 3000, 38 @ 800 uA']
dataFdict5.columns = ['2 @ 3000 uA, 38 @ 1250 uA','2 @ 3000 uA']


dataMelt4 = pd.melt(dataFdict4)
dataMelt4['Subject'] = '2'
dataMelt4['block'] = '1'
               
dataMelt5 = pd.melt(dataFdict5)
dataMelt5['Subject'] = '2'
dataMelt5['block'] = '2'


### now clean them all up
dataMelt1.rename(columns = {'variable':'experiment','value':'response time (ms)'}, inplace = True)
dataMelt2.rename(columns = {'variable':'experiment','value':'response time (ms)'}, inplace = True)
dataMelt3.rename(columns = {'variable':'experiment','value':'response time (ms)'}, inplace = True)
dataMelt4.rename(columns = {'variable':'experiment','value':'response time (ms)'}, inplace = True)
dataMelt5.rename(columns = {'variable':'experiment','value':'response time (ms)'}, inplace = True)


In [30]:
### concatenate

frames = [dataMelt1,dataMelt2,dataMelt3,dataMelt4,dataMelt5]

result = pd.concat(frames)
result
resultTrim = result[(result['response time (ms)'] < respHi) & (result['response time (ms)'] > respLo)]
resultTrim.groupby(['experiment','Subject','block']).mean()

response time (ms)
experiment                Subject block                    
1000 uA 200 ms            1       1              843.939840
1000 uA 300 ms            1       1              638.115840
1250 uA                   2       1              350.954732
1500 uA 200 ms            1       1              440.967168
2 @ 3000 uA               2       2              411.486569
2 @ 3000 uA, 38 @ 1250 uA 2       1              362.606833
                                  2              284.231078
2 @ 3000, 38 @ 800 uA     2       1              468.061623
2000 uA 100 ms            1       1              401.504256
3000 uA                   2       1              271.128576
constant train            1       2              511.185920
                                  3              457.850880
haptic                    1       1              315.484160
                          2       1              190.303573
high initial pulses       1       2              463.564800
                                  3              432.411569
off-target                1       1              439.336960

In [ ]:
### look at various metrics of interest


dataCleaned = resultTrim[resultTrim['experiment'].isin(experimentsOfInt)]
dataCleaned.groupby(['experiment','Subject','block']).mean()
dataCleaned.groupby(['experiment','Subject','block']).describe()
dataCleaned.groupby(['Subject','experiment']).describe().round()

#dataCleaned.groupby(['Subject','experiment']).mean()
#dataCleaned.groupby(['Subject','experiment']).std()

# get counts of each type
#dataCleaned.groupby(['Subject','experiment']).count()